# io of different FE data

* VMAP io
* Ansys io
* Abaqus io (still in progress)

In [ ]:
import numpy as np
import pandas as pd
import pylife.vmap
import pylife.stress.equistress
import pylife.mesh.gradient
import pyvista as pv
from ansys.mapdl import reader as pymapdl_reader

### VMAP

In [ ]:
vm_mesh = pylife.vmap.VMAPImport("plate_with_hole.vmap")

In [ ]:
pyLife_mesh = (vm_mesh.make_mesh('1', 'STATE-2')
               .join_coordinates()
               .join_variable('STRESS_CAUCHY')
               .join_variable('DISPLACEMENT')
               .to_frame())
pyLife_mesh.sample(5)

In [ ]:
pyLife_mesh['mises'] = pyLife_mesh.equistress.mises()
pyLife_mesh.sample(5)

We are using Pyvista for plotting FEM 

still work in progress, sorry dudes

In [ ]:
pyLife_nodes = pyLife_mesh.groupby('node_id').mean()
mesh = pv.PolyData(pyLife_nodes[['x', 'y', 'z']].values)
mesh.point_arrays["mises"] = pyLife_nodes["mises"].values
# mesh.plot(scalars="mises")

or using itk (for more information see here: https://pyvista-doc.readthedocs.io/en/latest/plotting/itk_plotting.html)

In [ ]:
# pl = pv.PlotterITK()
# pl.add_mesh(mesh, scalars="mises", smooth_shading=True)
# pl.show(True)
pyLife_mesh.columns

#### output to vmap

you can export vmap data easily to read in this data into Ansys, Abaqus and so on

In [ ]:
vmap_out = pylife.vmap.VMAPExport("vmap_export.vmap")
vmap_out.add_geometry("GEOMETRY_1", pyLife_mesh)
vmap_out.add_variable("2","GEOMETRY_1","mises",pyLife_mesh,["mises"],2)

In [ ]:
import_test = pylife.vmap.VMAPImport("vmap_export.vmap")
import_test.nodes('GEOMETRY_1')
meshimp=import_test.make_mesh('GEOMETRY_1')
meshimp.join_coordinates().join_variable('mises', state="2", column_names=["mises"])
meshimp.to_frame()

### ANSYS

In [ ]:
result = pymapdl_reader.read_binary('beam.rst')
elem_df = pd.DataFrame(result.mesh.elem).iloc[:,10:]
elem_df["element_id"] = pd.DataFrame(result.mesh.elem).iloc[:,8]
index_mesh = pd.DataFrame(index=pd.MultiIndex.from_frame(
    pd.melt(elem_df, id_vars="element_id", value_name="node_id").drop("variable",axis=1)))
nnum, stress = result.nodal_stress(0)
ansys_mesh = pd.DataFrame(data=np.concatenate((result.mesh.nodes,stress), axis=1),
                          index=nnum,
                          columns=['x', 'y', 'z', 'S11', 'S22', 'S33', 'S12', 'S23', 'S13']).dropna()
ansys_mesh.index.name = "node_id"
ansys_df = index_mesh.join(ansys_mesh).dropna()
display(ansys_df)

In [ ]:
# calculate the stress gradient
ansys_df['mises'] = ansys_df.equistress.mises()

In [ ]:
grad = ansys_df.gradient.gradient_of('mises')
grad["gradient_abs"]=np.linalg.norm(grad, axis=1)
ansys_df = ansys_df.join(grad)
ansys_df

In [ ]:
# You can use the pyansys API or working directly with the pyLife dataframe as above
result.plot_principal_nodal_stress(0, 'SEQV')

In [ ]:
# or you can use the pyLife-vtk API
nodes = ansys_df.groupby('node_id').mean()
mesh = pv.PolyData(nodes[['x', 'y', 'z']].values)
mesh.point_arrays["gradient_abs"] = np.log10(nodes["gradient_abs"].values)
mesh.plot(scalars="gradient_abs")

In [ ]:
pl = pv.PlotterITK()
pl.add_mesh(mesh, scalars="gradient_abs", smooth_shading=True)
pl.show(True)

In [ ]:
nodes["gradient_abs"]